# WEEK 4 Exercises - Making Queries

![ERD-E-COMMERCE](./WEEK3-ERD(e-commerce).png)

**ต่อเนื่องจากแบบฝึกหัดใน Week 3 ให้นักศึกษา run ไฟล์ `shop.sql` เพื่อเป็นการเพิ่มข้อมูลลงใน database**

**Hint:** สามารถใช้ project และ database ที่ได้ทำไว้ใน Week 3 ได้เลยครับ

ให้นักศึกษาสร้างโปรเจคใหม่ใหม่ชื่อ `myshop`

จากนั้นให้ทำการ startapp ใหม่ชื่อ `shop`

ให้นักศึกษาทำการเพิ่ม model นี้ในไฟล์ shop/models.py

In [1]:
from django.db import models

# Create your models here.


class Customer(models.Model):
    first_name = models.CharField(max_length=150)
    last_name = models.CharField(max_length=200)
    email = models.CharField(max_length=150)
    address = models.JSONField(null=True)

class ProductCategory(models.Model):
    name = models.CharField(max_length=150)

class Product(models.Model):
    name = models.CharField(max_length=150)
    description = models.TextField(null=True, blank=True)
    remaining_amount = models.PositiveIntegerField(default=0)
    price = models.DecimalField(max_digits=10, decimal_places=2)
    categories = models.ManyToManyField(ProductCategory)

class Cart(models.Model):
    customer = models.ForeignKey(Customer, on_delete=models.CASCADE)
    create_date = models.DateTimeField()
    expired_in = models.PositiveIntegerField(default=60)
    
class CartItem(models.Model):
    cart = models.ForeignKey(Cart, on_delete=models.CASCADE)
    product = models.ForeignKey(Product, on_delete=models.CASCADE)
    amount = models.PositiveIntegerField(default=1)
    
class Order(models.Model):
    customer = models.ForeignKey(Customer, on_delete=models.CASCADE)
    order_date = models.DateField()
    remark = models.TextField(null=True, blank=True)

class OrderItem(models.Model):
    order = models.ForeignKey(Order, on_delete=models.CASCADE)
    product = models.ForeignKey(Product, on_delete=models.CASCADE)
    amount = models.PositiveIntegerField(default=1)
    
class Payment(models.Model):
    order = models.OneToOneField(Order, on_delete=models.PROTECT)
    payment_date = models.DateField()
    remark = models.TextField(null=True, blank=True)
    price = models.DecimalField(max_digits=10, decimal_places=2)
    discount = models.DecimalField(max_digits=10, decimal_places=2, default=0)

class PaymentItem(models.Model):
    payment = models.ForeignKey(Payment, on_delete=models.CASCADE)
    order_item = models.OneToOneField(OrderItem, on_delete=models.CASCADE)
    price = models.DecimalField(max_digits=10, decimal_places=2)
    discount = models.DecimalField(max_digits=10, decimal_places=2, default=0)
    
class PaymentMethod(models.Model):
    class MethodChoices(models.Choices):
        QR = "QR"
        CREDIT = "CREDIT"
    
    payment = models.ForeignKey(Payment, on_delete=models.CASCADE)
    method = models.CharField(max_length=15, choices=MethodChoices.choices)
    price = models.DecimalField(max_digits=10, decimal_places=2)

ImproperlyConfigured: Requested setting INSTALLED_APPS, but settings are not configured. You must either define the environment variable DJANGO_SETTINGS_MODULE or call settings.configure() before accessing settings.

จากนั้นให้ทำการ migrate และ run คำสั่งในไฟล์ shop.sql

In [1]:
# ตั้ง env ให้ใช้กับ project django
import os
import django
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ['DJANGO_SETTINGS_MODULE'] = 'myshop.settings'  
django.setup()


### 1. ให้นักศึกษา Query ค้นหาข้อมูลมาแสดงให้ถูกต้องตามโจทย์

1.1 query หาข้อมูล `Order` ทั้งหมดที่เกิดขึ้นในเดือน `พฤษภาคม` และ`ราคา`ของ Order มาแสดงผล 10 รายการแรก และแสดงผลดังตัวอย่าง (0.5 คะแนน)

```txt
ORDER ID:22, DATE: 2024-05-01, PRICE: 4890.00
ORDER ID:23, DATE: 2024-05-01, PRICE: 2540.00
ORDER ID:24, DATE: 2024-05-01, PRICE: 1720.00
ORDER ID:25, DATE: 2024-05-02, PRICE: 322499.00
ORDER ID:26, DATE: 2024-05-02, PRICE: 3399.00
ORDER ID:27, DATE: 2024-05-02, PRICE: 1190.00
ORDER ID:28, DATE: 2024-05-03, PRICE: 9499.00
ORDER ID:29, DATE: 2024-05-03, PRICE: 700.00
ORDER ID:30, DATE: 2024-05-03, PRICE: 1690.00
ORDER ID:31, DATE: 2024-05-04, PRICE: 3280.00
```

In [2]:
#วิธี 1

# Django ORM = Object Relational Mapping คือเขียน class/object ของไพธอนในการติดต่อฐานข้อมูล
from shop.models import PaymentMethod, Payment, PaymentItem, OrderItem, Order, Cart, CartItem, ProductCategory, Product, Customer

# [:10] คือ LIMIT 10 (__ underscore 2 ครั้ง = การใช้คำสั่งของ django ได้)
order_in_may = Order.objects.filter(order_date__month=5)[:10]

# print(order_in_may.query) ดู query

# select_related คือ INNER JOIN (สำหรับความสัมพันธ์แบบ 1-1 กับ 1-M)
# 1-1 fliter จากฝั่งไหนก็ได้
orders_with_payment = Payment.objects.filter(order__in=order_in_may).select_related('order')

# เพิ่มเติม prefetch_related() คือ INNER JOIN (สำหรับความสัมพันธ์แบบ M-M)
# print(orders_with_payment.query) ดู query

# ใช้ loop เพื่อ print Object ออกมา
for payment in orders_with_payment:
    print(f"ORDER ID: {payment.order.id}, DATE: {payment.order.order_date} ,PRICE: {payment.price}")


ORDER ID: 22, DATE: 2024-05-01 ,PRICE: 4890.00
ORDER ID: 23, DATE: 2024-05-01 ,PRICE: 2540.00
ORDER ID: 24, DATE: 2024-05-01 ,PRICE: 1720.00
ORDER ID: 25, DATE: 2024-05-02 ,PRICE: 322499.00
ORDER ID: 26, DATE: 2024-05-02 ,PRICE: 3399.00
ORDER ID: 27, DATE: 2024-05-02 ,PRICE: 1190.00
ORDER ID: 28, DATE: 2024-05-03 ,PRICE: 9499.00
ORDER ID: 29, DATE: 2024-05-03 ,PRICE: 700.00
ORDER ID: 30, DATE: 2024-05-03 ,PRICE: 1690.00
ORDER ID: 31, DATE: 2024-05-04 ,PRICE: 3280.00


In [ ]:
# Simple 
order_may = Order.objects.filter(order_date__month = 5)[:10]

for order in order_may:
    print(f"ORDER ID:{order.id}, DATE: {order.order_date}, PRICE: {order.payment.price} ")

In [17]:
#วิธี 2

# Django ORM = Object Relational Mapping คือเขียน class/object ของไพธอนในการติดต่อฐานข้อมูล
from shop.models import PaymentMethod, Payment, PaymentItem, OrderItem, Order, Cart, CartItem, ProductCategory, Product, Customer

# [:10] คือ LIMIT 10 (__ underscore 2 ครั้ง = การใช้คำสั่งของ django ได้)
order_in_may = Order.objects.filter(order_date__month=5)[:10]

# print(order_in_may.query) ดู query

# ใช้ loop เพื่อ print Object ออกมา
for payment in orders_with_payment:
    #ใช้ loop เพื่อเอาออกมาให้เพราะ model มีการเชื่อมการอยู่แล้ว
    print(f"ORDER ID: {payment.order.id}, DATE: {payment.order.order_date} ,PRICE: {payment.price}")

ORDER ID: 22, DATE: 2024-05-01 ,PRICE: 4890.00
ORDER ID: 23, DATE: 2024-05-01 ,PRICE: 2540.00
ORDER ID: 24, DATE: 2024-05-01 ,PRICE: 1720.00
ORDER ID: 25, DATE: 2024-05-02 ,PRICE: 322499.00
ORDER ID: 26, DATE: 2024-05-02 ,PRICE: 3399.00
ORDER ID: 27, DATE: 2024-05-02 ,PRICE: 1190.00
ORDER ID: 28, DATE: 2024-05-03 ,PRICE: 9499.00
ORDER ID: 29, DATE: 2024-05-03 ,PRICE: 700.00
ORDER ID: 30, DATE: 2024-05-03 ,PRICE: 1690.00
ORDER ID: 31, DATE: 2024-05-04 ,PRICE: 3280.00


1.2 query หาข้อมูล `Product` ที่มีคำลงท้ายว่า `features.` ในรายละเอียดสินค้า และแสดงผลดังตัวอย่าง (0.5 คะแนน)

```txt
PRODUCT ID: 1, DESCRIPTION: A sleek and powerful smartphone with advanced features.
PRODUCT ID: 7, DESCRIPTION: High-resolution digital camera with advanced photography features.
PRODUCT ID: 10, DESCRIPTION: A stylish smartwatch with health monitoring and notification features.
PRODUCT ID: 14, DESCRIPTION: Split air conditioner with remote control and energy-saving features.
PRODUCT ID: 45, DESCRIPTION: Customizable racing track set with loop and jump features.
```

In [21]:
# Django ORM = Object Relational Mapping คือเขียน class/object ของไพธอนในการติดต่อฐานข้อมูล
from shop.models import PaymentMethod, Payment, PaymentItem, OrderItem, Order, Cart, CartItem, ProductCategory, Product, Customer


# endswith = LIKE %features. startswith = LIKE %features.
products = Product.objects.filter(description__endswith='features.')

# print(products.query)  ดู query

for product in products:
    print(f"PRODUCT ID: {product.id}, DESCRIPTION: {product.description}")

PRODUCT ID: 69, DESCRIPTION: Display Screen. 16.0
PRODUCT ID: 67, DESCRIPTION: By J. K. Rowling.
PRODUCT ID: 2, DESCRIPTION: A lightweight and high-performance laptop for professionals.
PRODUCT ID: 3, DESCRIPTION: An ultra-HD smart television with streaming capabilities.
PRODUCT ID: 4, DESCRIPTION: Wireless earphones with noise-canceling technology.
PRODUCT ID: 5, DESCRIPTION: A compact tablet for entertainment and productivity.
PRODUCT ID: 6, DESCRIPTION: A powerful gaming console with immersive graphics.
PRODUCT ID: 8, DESCRIPTION: High-speed wireless router for home or office use.
PRODUCT ID: 9, DESCRIPTION: Compact and portable power bank for charging devices on the go.
PRODUCT ID: 11, DESCRIPTION: Energy-efficient refrigerator with ample storage space.
PRODUCT ID: 12, DESCRIPTION: Front-loading washing machine with multiple wash programs.
PRODUCT ID: 13, DESCRIPTION: Compact microwave oven with defrost and cooking functions.
PRODUCT ID: 15, DESCRIPTION: Powerful vacuum cleaner for

1.3 query หาข้อมูล `Product` ที่มีราคาสินค้าตั้งแต่ `5000.00` ขึ้นไป และแสดงผลดังตัวอย่าง (0.5 คะแนน)

```txt
PRODUCT ID: 1, NAME: Smartphone, PRICE: 5900.00
PRODUCT ID: 2, NAME: Laptop, PRICE: 25999.00
PRODUCT ID: 3, NAME: Smart TV, PRICE: 8900.00
PRODUCT ID: 5, NAME: Tablet, PRICE: 12900.00
PRODUCT ID: 6, NAME: Gaming Console, PRICE: 5000.00
PRODUCT ID: 7, NAME: Digital Camera, PRICE: 32000.00
PRODUCT ID: 11, NAME: Refrigerator, PRICE: 9000.00
PRODUCT ID: 14, NAME: Air Conditioner, PRICE: 18900.00
PRODUCT ID: 31, NAME: Sofa, PRICE: 7000.00
PRODUCT ID: 54, NAME: Automatic Pet Feeder, PRICE: 7900.00
PRODUCT ID: 61, NAME: Diamond Stud Earrings, PRICE: 320000.00
PRODUCT ID: 62, NAME: Silver Charm Bracelet, PRICE: 70000.00
PRODUCT ID: 63, NAME: Gold Pendant Necklace, PRICE: 59000.00
PRODUCT ID: 64, NAME: Gemstone Ring, PRICE: 9000.00
PRODUCT ID: 65, NAME: Rose Gold Hoop Earrings, PRICE: 1200000.00
```

In [ ]:
# 1.3 query หาข้อมูล Product ที่มีราคาสินค้าตั้งแต่ 5000.00 ขึ้นไป และอยู่ในหมวดหมู่ Information Technology และแสดงผลดังตัวอย่าง (0.5 คะแนน)

# ตัวอย่างบางส่วน 
# PRODUCT ID: 1, NAME: Smartphone, PRICE: 5900.00

#  categories__name คือ การไปยัง field name ของตาราง productcatagory (เชื่อมไปที่ตาราง productcatagory แล้ว)
products = Product.objects.filter(Q(price__gt = 5000), Q(categories__name = "Information Technology" ) )
for product in products:
    print(f"PRODUCT ID: {product.id}, NAME: {product.name}, PRICE: {product.price}")

In [23]:
# Django ORM = Object Relational Mapping คือเขียน class/object ของไพธอนในการติดต่อฐานข้อมูล
from shop.models import PaymentMethod, Payment, PaymentItem, OrderItem, Order, Cart, CartItem, ProductCategory, Product, Customer

#  __lte -> Less than or equal
# __gte -> Greater than or equal
# __lt -> Less than
#__gt -> Greater than
products = Product.objects.filter(price__gte=5000)

for product in products:
    print(f"PRODUCT ID: {product.id}, NAME: {product.name}, PRICE: {product.price}")



PRODUCT ID: 69, NAME: Notebook HP Pavilion Silver, PRICE: 20000.00
PRODUCT ID: 1, NAME: Smartphone, PRICE: 5900.00
PRODUCT ID: 2, NAME: Laptop, PRICE: 25999.00
PRODUCT ID: 3, NAME: Smart TV, PRICE: 8900.00
PRODUCT ID: 5, NAME: Tablet, PRICE: 12900.00
PRODUCT ID: 6, NAME: Gaming Console, PRICE: 5000.00
PRODUCT ID: 7, NAME: Digital Camera, PRICE: 32000.00
PRODUCT ID: 11, NAME: Refrigerator, PRICE: 9000.00
PRODUCT ID: 14, NAME: Air Conditioner, PRICE: 18900.00
PRODUCT ID: 31, NAME: Sofa, PRICE: 7000.00
PRODUCT ID: 54, NAME: Automatic Pet Feeder, PRICE: 7900.00
PRODUCT ID: 61, NAME: Diamond Stud Earrings, PRICE: 320000.00
PRODUCT ID: 62, NAME: Silver Charm Bracelet, PRICE: 70000.00
PRODUCT ID: 63, NAME: Gold Pendant Necklace, PRICE: 59000.00
PRODUCT ID: 64, NAME: Gemstone Ring, PRICE: 9000.00
PRODUCT ID: 65, NAME: Rose Gold Hoop Earrings, PRICE: 1200000.00


1.4 query หาข้อมูล `Product` ที่มีราคาสินค้าน้อยกว่า `200.00` และมากกว่า `100.00` และแสดงผลดังตัวอย่าง (0.5 คะแนน)

```txt
PRODUCT ID: 28, NAME: Women's Sweater, PRICE: 190.00
```

In [24]:
# Django ORM = Object Relational Mapping คือเขียน class/object ของไพธอนในการติดต่อฐานข้อมูล
from shop.models import PaymentMethod, Payment, PaymentItem, OrderItem, Order, Cart, CartItem, ProductCategory, Product, Customer
from django.db.models import Q

# ใช้ Q objects เพื่อ query แบบ logic ได่้ (AND, OR, NOT)
products = Product.objects.filter(Q(price__lt=200) & Q(price__gt=100))
# products = Product.objects.filter(price__range=(100, 200))

for product in products:
    print(f"PRODUCT ID: {product.id}, NAME: {product.name}, PRICE: {product.price}")

PRODUCT ID: 21, NAME: Men's T-Shirt, PRICE: 200.00
PRODUCT ID: 28, NAME: Women's Sweater, PRICE: 190.00
PRODUCT ID: 29, NAME: Unisex Cap, PRICE: 200.00
PRODUCT ID: 49, NAME: Baby Einstein Take Along Tunes Musical Toy, PRICE: 200.00


### 2. เพิ่ม ลบ แก้ไข สินค้า

#### หมวดหมู่สินค้า
- Information technology
- Electronics
- Clothing and Apparel
- Home Appliances
- Furniture
- Toys and Games
- Books and Media
- Pet Supplies
- Jewelry

In [14]:
# Django ORM = Object Relational Mapping คือเขียน class/object ของไพธอนในการติดต่อฐานข้อมูล
from shop.models import PaymentMethod, Payment, PaymentItem, OrderItem, Order, Cart, CartItem, ProductCategory, Product, Customer

categories = ProductCategory.objects.all()

print("หมวดหมู่สินค้า")
for category in categories:
    print(f' • {category.name}')

หมวดหมู่สินค้า
 • Information Technology
 • Electronics
 • Clothing and Apparel
 • Home Appliances
 • Furniture
 • Toys and Games
 • Books and Media
 • Pet Supplies
 • Jewelry


2.1 ให้เพิ่มสินค้าใหม่จำนวน 3 รายการ (0.5 คะแนน)

```txt
สินค้าที่ 1
ชื่อ: Philosopher's Stone (1997)
หมวดหมู่สินค้า: Books and Media
จำนวนคงเหลือ: 20
รายละเอียดซ: By J. K. Rowling.
ราคา: 790

สินค้าที่ 2
ชื่อ: Me Before You
หมวดหมู่สินค้า: Books and Media
จำนวนคงเหลือ: 40
รายละเอียดซ: A romance novel written by Jojo
ราคา: 390

สินค้าที่ 3
ชื่อ: Notebook HP Pavilion Silver
หมวดหมู่สินค้า: Information technology และ Electronics
จำนวนคงเหลือ: 10
รายละเอียดซ: Display Screen. 16.0
ราคา: 20000
```

In [26]:
# Django ORM = Object Relational Mapping คือเขียน class/object ของไพธอนในการติดต่อฐานข้อมูล
from shop.models import PaymentMethod, Payment, PaymentItem, OrderItem, Order, Cart, CartItem, ProductCategory, Product, Customer

# get = การเรียกข้อมูลที่มีอยู่แล้วในฐานข้อมูลมาใช้
books_and_media = ProductCategory.objects.get(name='Books and Media')
information_technology = ProductCategory.objects.get(name='Information Technology')
electronics = ProductCategory.objects.get(name='Electronics')


product1 = Product.objects.create(
    name="Philosopher's Stone (1997)",
    description="By J. K. Rowling.",
    price=790,
    remaining_amount=20
)
product1.categories.add(books_and_media)

product2 = Product.objects.create(
    name="Me Before You",
    description="A romance novel written by Jojo",
    price=390,
    remaining_amount=40
)
product2.categories.add(books_and_media)

product3 = Product.objects.create(
    name="Notebook HP Pavilion Silver",
    description="Display Screen. 16.0",
    price=20000,
    remaining_amount=10
)
product3.categories.add(information_technology, electronics)

2.2 แก้ไขชื่อสินค้า จาก `Philosopher's Stone (1997)` เป็น `Half-Blood Prince (2005)` (0.5 คะแนน)

In [27]:
# Django ORM = Object Relational Mapping คือเขียน class/object ของไพธอนในการติดต่อฐานข้อมูล
from shop.models import PaymentMethod, Payment, PaymentItem, OrderItem, Order, Cart, CartItem, ProductCategory, Product, Customer

product_get = Product.objects.get(pk=70)
product_get.name = "Half-Blood Prince (2005)"
product_get.save()
print(product_get.name)


Half-Blood Prince (2005)


2.3 แก้ไขชื่อหมวดหมู่สินค้า จาก `Books and Media` เป็น `Books` (0.5 คะแนน)

In [29]:
# Django ORM = Object Relational Mapping คือเขียน class/object ของไพธอนในการติดต่อฐานข้อมูล
from shop.models import PaymentMethod, Payment, PaymentItem, OrderItem, Order, Cart, CartItem, ProductCategory, Product, Customer

product_catagories_get = ProductCategory.objects.get(pk=7)
product_catagories_get.name = "Books"
product_catagories_get.save()
print(product_catagories_get)

Books and Media


2.4 ลบสินค้าทุกตัวที่อยู่ในหมวดหมู่ `Books` (0.5 คะแนน)

In [30]:
# Django ORM = Object Relational Mapping คือเขียน class/object ของไพธอนในการติดต่อฐานข้อมูล
from shop.models import PaymentMethod, Payment, PaymentItem, OrderItem, Order, Cart, CartItem, ProductCategory, Product, Customer

# get(name='Books') เพื่อดึงข้อมูลวัตถุหนึ่งที่ตรงกับเงื่อนไขที่ชื่อว่า Books
books_category = ProductCategory.objects.get(name='Books')

# ลบข้อมูลออกจาก database สามารถทำได้โดยใช้ method delete() แบบลบทีละหลายตัว
products_delete = Product.objects.filter(categories=books_category).delete()

# เพิ่มเติม

- `.get()` when you want to get a single unique object (ใช้กับ pk ดีสุด) คืนค่าเป็น Obj.
  
- `.fliter()` when you want to get all objects that match your lookup parameters. (เหมือนกับ Where ใน SQl) คืนค่าเป็น Query Set
  
- `objects.raw()`  manager method can be used to perform raw SQL queries 

- `exclude()` เป็น QuerySet method ที่ใช้ในการกรองผลลัพธ์ที่ไม่ต้องการออกจาก QuerySet (ตรงข้าม fliter)